In [1]:
!git clone https://github.com/LizaDerb/probing_compositionality/

Cloning into 'probing_compositionality'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 112 (delta 25), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (112/112), 12.21 MiB | 4.70 MiB/s, done.
Resolving deltas: 100% (25/25), done.


# Russian AN и NN

## Датасет

In [2]:
import pandas as pd

In [3]:
corpus_AN = pd.read_csv('/content/probing_compositionality/data/rus/an_corpus.csv')
corpus_NN = pd.read_csv('/content/probing_compositionality/data/rus/nn_corpus.csv')

Лейблы:

*   Non-compositional (0)
*   Compositional (1)
*   Ambiguous (2)

Каждое выражение относится только к одной из групп.

In [4]:
corpus_AN.groupby('label')['idiom'].value_counts()

label  idiom                 
0      новый тип                 16
       налоговый кодекс          12
       уголовный кодекс          10
       абсолютное большинство    10
       огненная трасса           10
                                 ..
2      групповой отбор            3
       давняя пора                3
       двойные стандарты          3
       длинный стол               3
       должный инструмент         3
Name: count, Length: 5591, dtype: int64

In [5]:
corpus_AN['label'].value_counts()

label
1    26548
0     1509
2     1309
Name: count, dtype: int64

In [6]:
corpus_NN.groupby('label')['idiom'].value_counts()

label  idiom            
0      точка зрения         220
       образ жизни           37
       рынок труда           24
       свобода слова         21
       душа населения        19
                           ... 
2      зона отдыха            2
       дележ пирога           2
       двор хана              2
       движение тел           2
       эфир телекомпании      2
Name: count, Length: 7305, dtype: int64

In [7]:
corpus_NN['label'].value_counts()

label
1    22846
0      785
2      308
Name: count, dtype: int64

In [8]:
corpus = pd.concat([corpus_NN, corpus_AN])

Исключим ambiguous

In [9]:
corpus = corpus[corpus['label'] != 2]

In [10]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51688 entries, 308 to 28056
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   idiom     51688 non-null  object
 1   label     51688 non-null  int64 
 2   sentence  51688 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.6+ MB


Вручную разделим датасет на test и train, чтобы выражения в выборках не повторялись.

In [11]:
corpus[corpus['label'] == 0]['idiom'].value_counts()[:300].sum()

2130

In [12]:
non_compos = corpus[corpus['label'] == 0]['idiom'].value_counts().index[:300].tolist()
print(non_compos)

['точка зрения', 'образ жизни', 'рынок труда', 'свобода слова', 'душа населения', 'повестка дня', 'новый тип', 'банк крови', 'ремень безопасности', 'вертикаль власти', 'культ личности', 'налоговый кодекс', 'огненная трасса', 'абсолютное большинство', 'уголовный кодекс', 'бассейн реки', 'свобода выбора', 'властелин колец', 'лунная программа', 'конец света', 'новая волна', 'крайний случай', 'земной шар', 'золотой запас', 'католическая церковь', 'квадратный метр', 'ключевой вопрос', 'комсомольский правда', 'королевский институт', 'костный мозг', 'красная планета', 'красная армия', 'крепостное право', 'круглый стол', 'личный состав', 'заёмные средства', 'львиная доля', 'международный арена', 'мировая судья', 'мобильная связь', 'нефтяной картель', 'новая газета', 'здоровый человек', 'гуманитарная помощь', 'железная дорога', 'временная администрация', 'боевые действия', 'большая пятерка', 'большая деньги', 'большое значение', 'бюджетный кодекс', 'великая революция', 'великий пост', 'вечная м

In [13]:
corpus[corpus['label'] == 1]['idiom'].value_counts()[:300].sum()

6444

In [14]:
compos = corpus[corpus['label'] == 1]['idiom'].value_counts().index[:300].tolist()
print(compos)

['заводская лаборатория', 'задняя поверхность', 'бывший министр', 'чемпионат мира', 'налоговый орган', 'вид спорта', 'российский медик', 'темп роста', 'тысяча рублей', 'рост цен', 'важный этап', 'крошечный станок', 'ультразвуковое колебание', 'конституционное собрание', 'городское поселение', 'доктор наук', 'решение проблем', 'тысяча человек', 'нештатная ситуация', 'миллион рублей', 'сахалинская область', 'система образования', 'тысяча долларов', 'незнакомый человек', 'длинная история', 'глава государства', 'миллион долларов', 'орган власти', 'энергетический кризис', 'круглосуточный режим', 'общее количество', 'большая потеря', 'миллиард рублей', 'миллиард долларов', 'друг друга', 'медицинская организация', 'нужный компонент', 'право человека', 'социалистический плюрализм', 'конкретная группа', 'единый учебник', 'основная группа', 'длинная надпись', 'кожаный ботинок', 'производительность труда', 'чемпион мира', 'принятие решений', 'современная бионика', 'решение задач', 'ленинская конц

In [15]:
phrase_train = non_compos[:150] + compos[:150]
phrase_test = non_compos[150:] + compos[150:]

In [16]:
corpus_train = corpus[corpus['idiom'].isin(phrase_train)]
corpus_test = corpus[corpus['idiom'].isin(phrase_test)]

In [17]:
print('train: ', len(corpus_train))
print('test: ', len(corpus_test))

train:  5907
test:  2667


## RuBERT

In [18]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [19]:
from datasets import Dataset

In [20]:
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
import warnings
import random
import pickle

import torch
from torch.utils.data import DataLoader

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn import preprocessing
from transformers import AutoTokenizer, AutoModel

In [21]:
train = Dataset.from_pandas(corpus_train)
test = Dataset.from_pandas(corpus_test)

In [22]:
train

Dataset({
    features: ['idiom', 'label', 'sentence', '__index_level_0__'],
    num_rows: 5907
})

In [23]:
test

Dataset({
    features: ['idiom', 'label', 'sentence', '__index_level_0__'],
    num_rows: 2667
})

In [24]:
rubert_tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
rubert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
torch.cuda.is_available()

True

In [25]:
device_cuda = torch.device("cuda")
device_cpu = torch.device("cpu")

In [26]:
def vectorize_dataset(data, tokenizer, model):
    model = model.to(device_cuda)
    res_cls = []
    res_mean = []
    for batch in tqdm(data):
        toks = tokenizer(batch['sentence'], padding='max_length', truncation=True, return_tensors='pt', max_length=300)
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in toks.items()})
        res_cls.append(model_output.last_hidden_state[:,0,:]) #достаём вектора для CLS токенов
        res_mean.append(torch.mean(model_output.last_hidden_state, dim=1)) #среднее
    res_cls = torch.vstack(res_cls)
    res_mean = torch.vstack(res_mean)
    return res_cls.detach().cpu().numpy(), res_mean.detach().cpu().numpy()

In [28]:
dl_train = DataLoader(train, batch_size=256, shuffle=False,
                drop_last=False)
dl_test = DataLoader(test, batch_size=256, shuffle=False,
                drop_last=False)
vecs_train_cls, vecs_train_mean = vectorize_dataset(dl_train, rubert_tokenizer, rubert)
vecs_test_cls, vecs_test_mean = vectorize_dataset(dl_test, rubert_tokenizer, rubert)

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
vectors = {'vecs_train_cls' : vecs_train_cls, 'vecs_train_mean' : vecs_train_mean,
           'vecs_test_cls' : vecs_test_cls, 'vecs_test_mean' : vecs_test_mean}

for name in vectors.keys():
    with open(f'{name}.pkl', 'wb') as file:
         pickle.dump(vectors[name], file)

In [34]:
vector_names = ['vecs_train_cls', 'vecs_train_mean', 'vecs_test_cls', 'vecs_test_mean']
vecs = []

for name in vector_names:
    with open(f'{name}.pkl', "rb") as fIn:
        vecs.append(pickle.load(fIn))

train_cls = vecs[0]
train_mean = vecs[1]
test_cls = vecs[2]
test_mean = vecs[3]

In [35]:
train_cls.shape

(5907, 768)

In [36]:
test_mean.shape

(2667, 768)

In [37]:
train_labels = train['label']
test_labels = test['label']

In [38]:
def calculate_metrics(true_labels, predictions):
    matrix = confusion_matrix(true_labels, predictions)
    acc = accuracy_score(true_labels, predictions)
    prec = precision_score(true_labels, predictions)
    rec = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    auc_pred = roc_auc_score(true_labels, predictions)
    return matrix, acc, prec, rec, f1, auc_pred

In [39]:
def baseline_rand_vec(vecs):
    size = vecs.shape[1]
    num_vecs = vecs.shape[0]
    rand_vecs = []
    for _ in range(num_vecs):
        rand_vecs.append(np.random.uniform(-5.0826163,1.5603778,size))
    return rand_vecs

In [40]:
def baseline_rand_pred(vecs, labels):
    num_vecs = vecs.shape[0]
    rand_vecs = []
    rand_pred = np.array(range(num_vecs), dtype=int)
    for i in range(0, num_vecs):
        rand_pred[i] = random.choice(labels)
    return rand_pred

In [41]:
def rand_pred_func(vecs, labels, true_labels, n=50):
    results_rand_pred = []
    for _ in range(n):
        pred_labels = baseline_rand_pred(vecs, labels)
        results_rand_pred.append(calculate_metrics(true_labels, pred_labels))
    res_rand_pred = pd.DataFrame(results_rand_pred, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
    res_rand_pred.to_csv('res_rand_pred.csv')

In [42]:
def get_results(train_data, train_labels, test_data, test_labels):
    results_train = []
    results_test = []
    for n in tqdm(range(50)):
        clf = MLPClassifier(random_state=n, max_iter=300, verbose=1)
        clf.fit(train_data, train_labels)
        train_pred = clf.predict(train_data)
        test_pred = clf.predict(test_data)
        results_train.append(calculate_metrics(train_labels, train_pred))
        results_test.append(calculate_metrics(test_labels, test_pred))
    return results_train, results_test

In [43]:
labels = [0, 1]

In [44]:
rand_pred_func(test_cls, labels, test_labels)

In [45]:
rand_vecs = baseline_rand_vec(train_cls)
results_train_rand, results_test_rand = get_results(rand_vecs, train_labels, test_cls, test_labels)
res_test_rand = pd.DataFrame(results_test_rand, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
res_train_rand = pd.DataFrame(results_train_rand, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
res_test_rand.to_csv('res_test_rand_vec.csv')
res_train_rand.to_csv('res_train_rand_vec.csv')

  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 1, loss = 0.90072377
Iteration 2, loss = 0.59728084
Iteration 3, loss = 0.56388843
Iteration 4, loss = 0.55266242
Iteration 5, loss = 0.52975059
Iteration 6, loss = 0.50522646
Iteration 7, loss = 0.47753541
Iteration 8, loss = 0.44207412
Iteration 9, loss = 0.40410638
Iteration 10, loss = 0.36709802
Iteration 11, loss = 0.32264648
Iteration 12, loss = 0.27509627
Iteration 13, loss = 0.23012934
Iteration 14, loss = 0.18859514
Iteration 15, loss = 0.15449287
Iteration 16, loss = 0.12473838
Iteration 17, loss = 0.09926087
Iteration 18, loss = 0.07781844
Iteration 19, loss = 0.06399440
Iteration 20, loss = 0.05003778
Iteration 21, loss = 0.04185704
Iteration 22, loss = 0.03441264
Iteration 23, loss = 0.02939737
Iteration 24, loss = 0.02487186
Iteration 25, loss = 0.02197254
Iteration 26, loss = 0.01863572
Iteration 27, loss = 0.01652935
Iteration 28, loss = 0.01499050
Iteration 29, loss = 0.01328174
Iteration 30, loss = 0.01185177
Iteration 31, loss = 0.01079386
Iteration 32, los

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Iteration 1, loss = 0.94396774
Iteration 2, loss = 0.62728052
Iteration 3, loss = 0.57918073
Iteration 4, loss = 0.55251550
Iteration 5, loss = 0.52565602
Iteration 6, loss = 0.49994610
Iteration 7, loss = 0.46421287
Iteration 8, loss = 0.43438089
Iteration 9, loss = 0.39455526
Iteration 10, loss = 0.35673201
Iteration 11, loss = 0.31558753
Iteration 12, loss = 0.27030218
Iteration 13, loss = 0.23024991
Iteration 14, loss = 0.19642886
Iteration 15, loss = 0.15986964
Iteration 16, loss = 0.12555693
Iteration 17, loss = 0.09999055
Iteration 18, loss = 0.08064195
Iteration 19, loss = 0.06411993
Iteration 20, loss = 0.05301729
Iteration 21, loss = 0.04303956
Iteration 22, loss = 0.03618218
Iteration 23, loss = 0.03056803
Iteration 24, loss = 0.02587496
Iteration 25, loss = 0.02246539
Iteration 26, loss = 0.01950434
Iteration 27, loss = 0.01755920
Iteration 28, loss = 0.01543810
Iteration 29, loss = 0.01369167
Iteration 30, loss = 0.01244997
Iteration 31, loss = 0.01129155
Iteration 32, los

In [46]:
results_train_mean, results_test_mean = get_results(train_mean, train_labels, test_mean, test_labels)
res_test_mean = pd.DataFrame(results_test_mean, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
res_train_mean = pd.DataFrame(results_train_mean, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
res_test_mean.to_csv('res_test_mean.csv')
res_train_mean.to_csv('res_train_mean.csv')

  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 1, loss = 0.55299757
Iteration 2, loss = 0.50358076
Iteration 3, loss = 0.47916139
Iteration 4, loss = 0.46127058
Iteration 5, loss = 0.44143490
Iteration 6, loss = 0.42018052
Iteration 7, loss = 0.40055906
Iteration 8, loss = 0.38249312
Iteration 9, loss = 0.36074465
Iteration 10, loss = 0.34164431
Iteration 11, loss = 0.32251215
Iteration 12, loss = 0.30255816
Iteration 13, loss = 0.28216087
Iteration 14, loss = 0.26410612
Iteration 15, loss = 0.24681734
Iteration 16, loss = 0.23246792
Iteration 17, loss = 0.21706546
Iteration 18, loss = 0.20678990
Iteration 19, loss = 0.19113502
Iteration 20, loss = 0.17937143
Iteration 21, loss = 0.16919695
Iteration 22, loss = 0.15773526
Iteration 23, loss = 0.14651361
Iteration 24, loss = 0.14150151
Iteration 25, loss = 0.13299433
Iteration 26, loss = 0.12829098
Iteration 27, loss = 0.11947586
Iteration 28, loss = 0.11229954
Iteration 29, loss = 0.10923778
Iteration 30, loss = 0.10372366
Iteration 31, loss = 0.10235288
Iteration 32, los

In [40]:
results_train_cls = []
results_test_cls = []

for n in range(50):
    clf = MLPClassifier(random_state=n, max_iter=300, verbose=1)
    clf.fit(train_cls, train_labels)
    train_pred = clf.predict(train_cls)
    test_pred = clf.predict(test_cls)
    results_train_cls.append(calculate_metrics(train_labels, train_pred))
    results_test_cls.append(calculate_metrics(test_labels, test_pred))

Выходные данные были обрезаны до нескольких последних строк (5000).
Iteration 38, loss = 0.37989683
Iteration 39, loss = 0.36745048
Iteration 40, loss = 0.37044026
Iteration 41, loss = 0.36427515
Iteration 42, loss = 0.35602873
Iteration 43, loss = 0.35368349
Iteration 44, loss = 0.34911546
Iteration 45, loss = 0.34647916
Iteration 46, loss = 0.33935272
Iteration 47, loss = 0.33294615
Iteration 48, loss = 0.33541733
Iteration 49, loss = 0.32720968
Iteration 50, loss = 0.33402729
Iteration 51, loss = 0.31794814
Iteration 52, loss = 0.31628391
Iteration 53, loss = 0.30993716
Iteration 54, loss = 0.30349722
Iteration 55, loss = 0.30317984
Iteration 56, loss = 0.29771257
Iteration 57, loss = 0.29003093
Iteration 58, loss = 0.29492629
Iteration 59, loss = 0.28445227
Iteration 60, loss = 0.27960914
Iteration 61, loss = 0.27571687
Iteration 62, loss = 0.27391417
Iteration 63, loss = 0.28002410
Iteration 64, loss = 0.28424262
Iteration 65, loss = 0.26337140
Iteration 66, loss = 0.26176308
Iter

In [41]:
res_test_cls = pd.DataFrame(results_test_cls, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
res_train_cls = pd.DataFrame(results_train_cls, columns=['matrix', 'accuracy', 'precision', 'recall', 'f1', 'auc_pred'])
res_test_cls.to_csv('res_test_cls.csv')
res_train_cls.to_csv('res_train_cls.csv')